# Data Compilation
First, compile all data. When clicking 'Compile Hikes', the code will combine all hikes stored in the repo, format it, and save it into one CSV.

In [163]:
from IPython.display import HTML


HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [167]:
from ipywidgets import widgets
import pandas as pd
import data_processing
from IPython.display import clear_output, display


def compile_hike_clicked(press):
    clear_output()
    display(data_button)
    data_processing.main()
file_name = 'complete_hikes_list'
df = pd.read_csv("files/{}.csv".format(file_name))
# df.head()
data_button = widgets.Button(description='Click to compile hikes')
display(data_button)
data_button.on_click(compile_hike_clicked)

Button(description=u'Click to compile hikes', style=ButtonStyle())

Processing: Blood Mountain
No corrected altitude provided for Blood Mountain
Processing: Brasstown Bald
No corrected altitude provided for Brasstown Bald
Processing: East Palisades
No corrected altitude provided for East Palisades
Processing: Sawnee
No corrected altitude provided for Sawnee
Processing: Sweetwater
No corrected altitude provided for Sweetwater
Saving complete list to './files/complete_hikes_list.csv'


In [169]:
import plotly.plotly as py
import cufflinks as cf
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt
# from ipywidgets import widgets
from IPython.display import clear_output, display

# def plot_graph(press):
#     clear_output()
#     display(controls)

elevation_df = df[['Delta Time', 'Name', 'Elevation Gain']]
elevation_df = elevation_df.pivot(index='Delta Time', columns='Name', values='Elevation Gain')
elevation_df.iplot(kind='scatter', connectgaps=True, filename='hike_elevation_gain', 
                   yTitle='Elevation Gain [m]', xTitle='Time [hh:mm]', title='Altitude climbed in every hike')
# elevation_df.head()


In [170]:
hike_name = 'Blood Mountain'
hike_data = '2018-04-11'
activity_name = 'heart'
value = 'Elevation Gain'

fitbit_file = '{0}_{1}_{2}'.format(hike_name, activity_name, hike_data)
fitbit_df = pd.read_csv("files/fitbit_data/{}.csv".format(fitbit_file), index_col=0).drop(['Time','Date'], axis=1)

activity_df = df[['Timestamp', 'Name', 'Elevation Gain']]
activity_df = activity_df.pivot(index='Timestamp', columns='Name', values='Elevation Gain')
activity_df = activity_df.loc[:,hike_name].reset_index().dropna()
activity_df.set_index('Timestamp', drop=True, inplace=True)
combined_df = pd.concat([fitbit_df, activity_df], sort=False)
combined_df = combined_df.rename(columns={hike_name:value}).sort_values('Timestamp')

data = [go.Scatter(x=combined_df.index, y=combined_df[value], name=value, 
                   line = dict(color = ('rgb(22, 96, 167)')),connectgaps=True),
        go.Scatter(x=combined_df.index, y=combined_df[activity_name.capitalize()], name='Heart rate', yaxis='y2',
                  line = dict(color = ('rgb(101, 204, 73)')),connectgaps=True)]

layout = go.Layout(title='Heart rate at {}'.format(hike_name),
                   # Define first yaxis
                   yaxis=dict(
                       title='Elevation Gain [m]'),
                   # Define second yaxis
                   yaxis2=dict(
                       title='Heart Rate [bpm]',
                       titlefont=dict(
                           color='gray'),
                       tickfont=dict(
                           color='gray'),
                       overlaying='y',
                       side='right')
                  )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='elevation_heartRate')

# combined_df.head()
